In [24]:
import pandas as pd
import json
from datetime import datetime

In [36]:
def flatten_list(nested_list):
    """ Aplatir récursivement une liste de listes """
    if isinstance(nested_list, list):
        flat_list = []
        for item in nested_list:
            if isinstance(item, list):
                flat_list.extend(flatten_list(item))  # Appel récursif pour déplier les sous-listes
            else:
                flat_list.append(item)
        return flat_list
    return nested_list

def json_to_dataframe(json_file):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f"{json_file}_{timestamp}.csv"

    # Charger le fichier JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Normalisation pour aplatir les objets imbriqués (JSON complexe)
    df = pd.json_normalize(data, sep='_')

    # Détection des colonnes contenant des listes
    list_columns = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, list)).any()]

    # Aplatir les listes imbriquées avant de faire explode()
    for col in list_columns:
        df[col] = df[col].apply(flatten_list)  # Appliquer le flattening
        df = df.explode(col)  # Déplier les listes

    # Aplatir les objets imbriqués restants après explosion
    for col in list_columns:
        if df[col].dropna().apply(lambda x: isinstance(x, dict)).any():
            nested_df = pd.json_normalize(df[col].dropna()).add_prefix(f"{col}_")
            df = df.drop(columns=[col]).reset_index(drop=True).join(nested_df.reset_index(drop=True))

    # Sauvegarde en CSV
    df.to_csv(filename, index=False)

    return df


In [37]:
json_file = "G1_2020.json"
df = json_to_dataframe(json_file)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9540 entries, 0 to 9539
Columns: 215 entries, id to earnings_detailedEarnings_earnings
dtypes: bool(6), float64(28), int64(8), object(173)
memory usage: 15.3+ MB


In [41]:
df.describe()

,raceNumber,distance,earnings_totalPool,raceType,trackType,ranking_rank,ranking_runnerNumber,runners_runnerNumber,runners_railPosition,runners_claimRate,...,runners_horse.sire.breed,runners_horse.dam.sire.breed,runners_horse.dam.origin,runners_horse.origin,runners_horse.dam.gender,runners_horse.breed,runners_horse.sire.origin,runners_horse.dam.sire,earnings_detailedEarnings_rank,earnings_detailedEarnings_earnings
count,9540.000000,9540.000000,9.540000e+03,0.0,0.0,9540.000000,9540.000000,9540.000000,9540.000000,9540.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9315.000000,9.315000e+03
mean,5.324948,2504.821803,1.312348e+06,NaN,NaN,5.865304,6.045597,6.116352,6.116352,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,2.661080e+05
std,1.779685,480.730282,1.952001e+06,NaN,NaN,4.045966,4.148354,4.176633,4.176633,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.414289,6.141848e+05
min,2.000000,2000.000000,3.300000e+04,NaN,NaN,1.000000,1.000000,1.000000,1.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.320000e+03
25%,4.000000,2400.000000,9.000000e+04,NaN,NaN,3.000000,3.000000,3.000000,3.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,9.100000e+03
50%,5.000000,2400.000000,1.300000e+05,NaN,NaN,5.000000,5.000000,5.000000,5.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,2.860000e+04
75%,7.000000,2500.000000,3.000000e+06,NaN,NaN,8.000000,8.000000,8.000000,8.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,1.430000e+05
max,12.000000,4000.000000,5.000000e+06,NaN,NaN,19.000000,19.000000,19.000000,19.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,2.857000e+06


In [42]:
df.head()

,id,metaData,meetingId,raceNumber,raceName,plannedStartTime,actualStartTime,complexBetCode,runnersCount,distance,...,runners_horse.dam.colour.label,runners_horse.dam.colour.language,runners_horse.dam.gait.code,runners_horse.dam.gait.label,runners_horse.dam.gait.language,runners_horse.breed,runners_horse.sire.origin,runners_horse.dam.sire,earnings_detailedEarnings_rank,earnings_detailedEarnings_earnings
0,46dee40e-f241-360f-a669-e8e2df8d5a56,None,20201129-TOK,12,THE JAPAN CUP (G1),07:40,None,None,None,2400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1714200.0
1,46dee40e-f241-360f-a669-e8e2df8d5a56,None,20201129-TOK,12,THE JAPAN CUP (G1),07:40,None,None,None,2400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,685800.0
2,46dee40e-f241-360f-a669-e8e2df8d5a56,None,20201129-TOK,12,THE JAPAN CUP (G1),07:40,None,None,None,2400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,342900.0
3,46dee40e-f241-360f-a669-e8e2df8d5a56,None,20201129-TOK,12,THE JAPAN CUP (G1),07:40,None,None,None,2400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,171300.0
4,46dee40e-f241-360f-a669-e8e2df8d5a56,None,20201129-TOK,12,THE JAPAN CUP (G1),07:40,None,None,None,2400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,85800.0


In [44]:
df_group_id= df.groupby("id").agg(list).reset_index()
df_group_id.head(5)

,id,metaData,meetingId,raceNumber,raceName,plannedStartTime,actualStartTime,complexBetCode,runnersCount,distance,...,runners_horse.dam.colour.label,runners_horse.dam.colour.language,runners_horse.dam.gait.code,runners_horse.dam.gait.label,runners_horse.dam.gait.language,runners_horse.breed,runners_horse.sire.origin,runners_horse.dam.sire,earnings_detailedEarnings_rank,earnings_detailedEarnings_earnings
0,127515a2-458e-3f6a-a2aa-74513c9ca05b,"[None, None, None, None, None, None, None, Non...","[20170820-DEA, 20170820-DEA, 20170820-DEA, 201...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[DARLEY PRIX KERGORLAY, DARLEY PRIX KERGORLAY,...","[17:16, 17:16, 17:16, 17:16, 17:16, 17:16, 17:...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[3000, 3000, 3000, 3000, 3000, 3000, 3000, 300...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.0, 2.0, 3.0, 4.0, 5.0, 1.0, 2.0, 3.0, 4.0, ...","[74100.0, 28600.0, 13650.0, 9100.0, 4550.0, 74..."
1,13237960-c920-3aea-8ce6-b6b6a0f71c59,"[None, None, None, None, None, None, None, Non...","[20181021-CHA, 20181021-CHA, 20181021-CHA, 201...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[PRIX DU CONSEIL DE PARIS, PRIX DU CONSEIL DE ...","[17:33, 17:33, 17:33, 17:33, 17:33, 17:33, 17:...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[2400, 2400, 2400, 2400, 2400, 2400, 2400, 240...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.0, 2.0, 3.0, 4.0, 5.0, 1.0, 2.0, 3.0, 4.0, ...","[2857000.0, 1143000.0, 571500.0, 285500.0, 143..."
2,19dcedec-0b76-31d5-b30e-f8b6e686d1e5,"[None, None, None, None, None, None, None, Non...","[20200913-LON, 20200913-LON, 20200913-LON, 202...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[QATAR PRIX FOY, QATAR PRIX FOY, QATAR PRIX FO...","[17:10, 17:10, 17:10, 17:10, 17:10, 17:10, 17:...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[2400, 2400, 2400, 2400, 2400, 2400, 2400, 240...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1.0, 2.0, 3.0, 4.0, 5.0, 1.0, 2.0, 3.0, 4.0, ...","[137136.0, 54864.0, 27432.0, 13704.0, 6864.0, ..."
3,3b273d12-3e1b-3bd1-a58a-0710eee364f7,"[None, None, None, None, None, None, None, Non...","[20190317-SAI, 20190317-SAI, 20190317-SAI, 201...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[PRIX EXBURY, PRIX EXBURY, PRIX EXBURY, PRIX E...","[17:35, 17:35, 17:35, 17:35, 17:35, 17:35, 17:...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non...","[2000, 2000, 2000, 2000, 2000, 2000, 2000, 20

In [54]:
df_group_id.to_excel("G1_2020_group_id.xlsx", index=False)

In [53]:
df_group_id.columns.info()

AttributeError: 'Index' object has no attribute 'info'

In [48]:
df_group_id = df_group_id.apply(lambda x: list(dict.fromkeys(x)))

TypeError: unhashable type: 'list'